In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
from helper import make_reversi_vec_env, SelfPlayEnv
from stable_baselines3 import PPO
from models import CustomActorCriticPolicyMLP
from players import RandomPlayer
from stable_baselines3.common.callbacks import EvalCallback

In [18]:
board_shape = 8
n_envs = 8
gamma = 0.99
ent_coef = 0.0
gae_lambda = 0.95
n_epochs = 10

In [19]:
# random_player_eval = RandomPlayer(board_shape=board_shape, flatten_action=False)

In [20]:
eval_env = make_reversi_vec_env(
    SelfPlayEnv, n_envs=1,
    env_kwargs={'board_shape': board_shape, 'LocalPlayer': RandomPlayer}
)

In [21]:
# eval_env.reset()

In [22]:
model_name = f'PPO_{board_shape}by{board_shape}_{gamma}_{gae_lambda}_{ent_coef}_{n_epochs}_{n_envs}_masked_mlp'
best_model_save_path = f'./models/{model_name}'
print(model_name)
print(best_model_save_path)

PPO_8by8_0.99_0.95_0.0_10_8_masked_mlp
./models/PPO_8by8_0.99_0.95_0.0_10_8_masked_mlp


In [23]:
# random_player = RandomPlayer(board_shape=board_shape, flatten_action=False)

In [24]:
env = make_reversi_vec_env(
    SelfPlayEnv, n_envs=n_envs,
    env_kwargs={'board_shape': board_shape, 'LocalPlayer': RandomPlayer}
)

In [25]:
model = PPO(
    CustomActorCriticPolicyMLP,
    env,
    verbose=0,
    tensorboard_log='../reversi/testing/',
    gamma=gamma,
    gae_lambda=gae_lambda,
    ent_coef=ent_coef,
    n_epochs=n_epochs
)

In [26]:
model.policy

CustomActorCriticPolicyMLP(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (mlp_extractor): CustomNetwork(
    (policy_net): Sequential(
      (0): Linear(in_features=64, out_features=64, bias=True)
      (1): ReLU()
    )
    (value_net): Sequential(
      (0): Linear(in_features=64, out_features=64, bias=True)
      (1): ReLU()
    )
  )
  (action_net): Linear(in_features=64, out_features=64, bias=True)
  (value_net): Linear(in_features=64, out_features=1, bias=True)
)

In [27]:
def get_valid_actions(board):
    return eval_env.envs[0].get_valid((board, 1)).reshape(-1)  
model.policy.set_get_valid_actions(get_valid_actions)

In [28]:
eval_callback = EvalCallback(
    eval_env = eval_env,
    eval_freq=5_000,
    n_eval_episodes=500,
    deterministic=True,
    verbose=1,
    best_model_save_path=best_model_save_path,
)

In [ ]:
model.learn(total_timesteps=int(1e10), tb_log_name=model_name, callback=[eval_callback])

Eval num_timesteps=40000, episode_reward=0.53 +/- 0.83
Episode length: 29.89 +/- 1.03
New best mean reward!
Eval num_timesteps=80000, episode_reward=0.61 +/- 0.77
Episode length: 29.98 +/- 0.57
New best mean reward!
Eval num_timesteps=120000, episode_reward=0.66 +/- 0.74
Episode length: 29.86 +/- 1.54
New best mean reward!
Eval num_timesteps=160000, episode_reward=0.72 +/- 0.67
Episode length: 29.89 +/- 1.15
New best mean reward!
Eval num_timesteps=200000, episode_reward=0.69 +/- 0.71
Episode length: 29.89 +/- 1.42
Eval num_timesteps=240000, episode_reward=0.73 +/- 0.66
Episode length: 29.81 +/- 1.47
New best mean reward!
Eval num_timesteps=280000, episode_reward=0.72 +/- 0.68
Episode length: 29.93 +/- 0.61
Eval num_timesteps=320000, episode_reward=0.78 +/- 0.61
Episode length: 29.92 +/- 0.55
New best mean reward!
Eval num_timesteps=360000, episode_reward=0.78 +/- 0.60
Episode length: 29.87 +/- 0.74
Eval num_timesteps=400000, episode_reward=0.72 +/- 0.68
Episode length: 29.83 +/- 1.21
